<a href="https://colab.research.google.com/github/kaiwei1016/ColabProjects/blob/main/Python/%E7%B6%B2%E8%B7%AF%E7%88%AC%E8%9F%B2_%E6%AF%8F%E5%AD%A3%E8%B2%A1%E5%8B%99%E5%A0%B1%E8%A1%A8%E8%B3%87%E8%A8%8A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# # 爬取每季財報資訊（輸入公司代號,年度,季別）

In [43]:
import requests
from io import StringIO
import pandas as pd

sid = '1101'  #選擇公司代號
year = '2022'  #選擇年度
season = '3'  #選擇季別
res = requests.get('https://mops.twse.com.tw/server-java/t164sb01?step=3&SYEAR='+year+'&file_name=tifrs-fr1-m1-ci-cr-'+sid+'-'+year+'Q'+season+'.html') 
res.encoding = 'big5'
dfs = pd.read_html(StringIO(res.text))  #轉成DataFrame
dfs[0]  #表格順序：[0]資產負債表 [1]綜合損益表 [2]現金流量表

資產負債表Balance Sheet                                                     \
               代號Code                               會計項目Accounting Title   
0                 NaN                                           資產Assets   
1                 NaN                                流動資產　Current assets   
2                1100                  現金及約當現金 Cash and cash equivalents   
3                1110  透過損益按公允價值衡量之金融資產－流動 Current financial assets a...   
4                1120  透過其他綜合損益按公允價值衡量之金融資產－流動 Current financial asse...   
..                ...                                                ...   
75               36XX                    非控制權益 Non-controlling interests   
76               3XXX                                  權益總額 Total equity   
77               3X2X               負債及權益總計　Total liabilities and equity   
78               3998  預收股款（權益項下）之約當發行股數　Equivalent issue shares of a...   
79               3999  母公司暨子公司所持有之母公司庫藏股股數（單位：股）　Number of shares in ...   

                                                                  
   2022年9月30日2022/9/30 2021年12月31日2021/12/31 2021年9月30日2021/9/30  
0                  NaN                   NaN                 NaN  
1                  NaN                   NaN                 NaN  
2             95367543              91121942            59958809  
3               859007                363813              335314  
4              5789040               6387543             6180994  
..                 ...                   ...                 ...  
75            20256450              21177340            19205882  
76           221670446             225654673           220920689  
77           458733599             441684559           401380637  
78                   0                     0                   0  
79             4000000               9060600             9060600  

[80 rows x 5 columns]

# # 一次爬取多間公司的財報+儲存檔案

In [46]:
import time
import os

if 'fs' not in os.listdir():  #創建資料夾
    os.mkdir('fs')
sid = ['1101','1102','1103'] # 要抓取財報的公司代號，例如1101,1102,1103...
for s in sid:  # 用迴圈爬取html檔
    res = requests.get('https://mops.twse.com.tw/server-java/t164sb01?step=3&SYEAR=2022&file_name=tifrs-fr1-m1-ci-cr-'+ s +'-2022Q3.html')
    res.encoding = 'big5'
    path = os.path.join('fs', s + '.html') #存檔路徑
    f = open(path, 'w', encoding='utf-8')  #寫入檔案
    f.write(res.text)
    f.close()
    print("已存取:",s)  #顯示存取的進度
    time.sleep(5)  #每隔5秒存取一支

dfs = []
for s in sid:  #用迴圈把檔案依序存到list
    path = os.path.join('fs', s + '.html')
    dfs.append(pd.read_html(path, encoding='utf-8'))

已存取: 1101
已存取: 1102
已存取: 1103


# 再選擇要查看哪間公司的財報資訊:

*   dfs[公司順序][表格順序]
*   表格順序：[0]資產負債表 [1]綜合損益表 [2]現金流量表








In [54]:
dfs[1][2]  

現金流量表Statements of Cash Flows  \
                          代號Code   
0                            NaN   
1                         A00010   
2                         A10000   
3                            NaN   
4                            NaN   
..                           ...   
83                          DDDD   
84                          EEEE   
85                        E00100   
86                        E00200   
87                        E00210   

                                                       \
                                 會計項目Accounting Title   
0   營業活動之現金流量－間接法Cash flows from (used in) operati...   
1   繼續營業單位稅前淨利（淨損）　Profit (loss) from continuing o...   
2                 本期稅前淨利（淨損）　Profit (loss) before tax   
3                                    調整項目　Adjustments   
4       收益費損項目 Adjustments to reconcile profit (loss)   
..                                                ...   
83  匯率變動對現金及約當現金之影響Effect of exchange rate changes...   
84  本期現金及約當現金增加（減少）數Net increase (decrease) in cas...   
85  期初現金及約當現金餘額Cash and cash equivalents at beginn...   
86  期末現金及約當現金餘額Cash and cash equivalents at end of...   
87  資產負債表帳列之現金及約當現金　Cash and cash equivalents repo...   

                                                                
   2022年1月1日至9月30日2022/1/1To9/30 2021年1月1日至9月30日2021/1/1To9/30  
0                            NaN                           NaN  
1                       14159440                      16381396  
2                       14159440                      16381396  
3                            NaN                           NaN  
4                            NaN                           NaN  
..                           ...                           ...  
83                       1751427                     (485,312)  
84                       9916476                       7493779  
85                      33450618                      25911732  
86                      43367094                      33405511  
87                      43367094                      33405511  

[88 rows x 4 columns]